In [1]:
from ioos_qartod.qc_tests.qc import range_check

/Users/lcampbell/.virtualenvs/work/lib/python2.7/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory
  stacklevel=stacklevel + 1):


## Collecting the data from a GLOS source

We're going to grab the data from the Toledo Low Service Pump Station that measures water quality parameters, sepcifically `ysi_turbidity`

In [ ]:
url = 'http://tds.glos.us/thredds/dodsC/glos/habs/lakes/ysi_turbidity/tollsps/tollsps.ncml'
from netCDF4 import Dataset
nc = Dataset(url)